In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [26]:

pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Gc_884/COMPINT'
nx=256
ny=384
ncomp=4


'''
pref = '/home/saberbud/Programs/Paper_RB_14bit_COMPINTs/CI16164/COMPINT'
nx=128
ny=192
ncomp=4
'''

'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'
nx=64
ny=96
ncomp=4
'''

print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Gc_884/COMPINT


In [27]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 256, 384, 4) , mean = 148.92459869384766 , std = 30.341251894658537 , Max = 213.0 , Min = 62.0 , type=float64
t_C_0 shape: (1, 256, 384) , mean = 101.207763671875 , std = 9.546029266215388 , Max = 140.0 , Min = 62.0 , type=float64
t_C_1 shape: (1, 256, 384) , mean = 162.92457071940103 , std = 12.244569445887624 , Max = 211.0 , Min = 119.0 , type=float64
t_C_2 shape: (1, 256, 384) , mean = 171.49837239583334 , std = 14.737577990741773 , Max = 213.0 , Min = 125.0 , type=float64
t_C_3 shape: (1, 256, 384) , mean = 160.06768798828125 , std = 10.83857214018827 , Max = 190.0 , Min = 127.0 , type=float64


In [28]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (256, 384) , mean = 32.06768798828125 , std = 10.83857214018827 , Max = 62 , Min = -1 , type=int8


In [29]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen98304
size198316
size263399
ratio1= 1.00012207031
ratio2= 0.644927978516


In [30]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 98304
mse = 0.0


In [31]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 256, 384, 4) , mean = 148.19680531819662 , std = 30.59434579154023 , Max = 231.0 , Min = 55.0 , type=float64
 channel0 shape: (256, 384) , mean = -25.897898356119793 , std = 11.923433049010626 , Max = 16 , Min = -73 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 64320 Bytes
ratio1= 1.00012207031
ratio2= 0.654296875
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 33.720235188802086 , std = 14.977835970354413 , Max = 89 , Min = -9 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 67790 Bytes
ratio1= 1.00012207031
ratio2= 0.689595540365
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 42.06707763671875 , std = 17.581036706203392 , Max = 103 , Min = -2 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 69944 Bytes
ratio1= 1.00012207031
ratio2= 0.711507161458
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 30.897806803385418 , std = 12.962800376262704 , Max = 69 , Min = 0 , type=int8
   olen

dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 36.883992513020836 , std = 16.993875350385007 , Max = 89 , Min = -6 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 73002 Bytes
ratio1= 1.00012207031
ratio2= 0.742614746094
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 45.96782430013021 , std = 20.675354120990875 , Max = 101 , Min = 0 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 76209 Bytes
ratio1= 1.00012207031
ratio2= 0.775238037109
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 33.841532389322914 , std = 15.13637462362884 , Max = 74 , Min = 0 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 70682 Bytes
ratio1= 1.00012207031
ratio2= 0.719014485677
dlen= 98304
mse = 0.0



9
  t_C shape: (1, 256, 384, 4) , mean = 153.3631134033203 , std = 35.461005341590955 , Max = 221.0 , Min = 63.0 , type=float64
 channel0 shape: (256, 384) , mean = -32.50847371419271 , std = 8.513870555481256 , Max = 11 , Min =

   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 69626 Bytes
ratio1= 1.00012207031
ratio2= 0.708272298177
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 26.842936197916668 , std = 23.533850766631573 , Max = 68 , Min = -4 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 64829 Bytes
ratio1= 1.00012207031
ratio2= 0.659474690755
dlen= 98304
mse = 0.0



16
  t_C shape: (1, 256, 384, 4) , mean = 148.46649424235025 , std = 29.933673975151514 , Max = 216.0 , Min = 72.0 , type=float64
 channel0 shape: (256, 384) , mean = -26.128438313802082 , std = 10.035383738384448 , Max = 5 , Min = -56 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 57562 Bytes
ratio1= 1.00012207031
ratio2= 0.58555094401
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 34.10783894856771 , std = 12.705816887912661 , Max = 83 , Min = 0 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 61290 Bytes
ratio1= 1.00012207031
ratio2= 0.623474121094
dlen= 9830

dlen= 98304
mse = 0.0



23
  t_C shape: (1, 256, 384, 4) , mean = 147.6949895222982 , std = 30.494832067709794 , Max = 219.0 , Min = 70.0 , type=float64
 channel0 shape: (256, 384) , mean = -25.18768310546875 , std = 12.105719898095295 , Max = 12 , Min = -58 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 66351 Bytes
ratio1= 1.00012207031
ratio2= 0.674957275391
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 32.88188680013021 , std = 15.679985798043129 , Max = 80 , Min = -11 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 70838 Bytes
ratio1= 1.00012207031
ratio2= 0.72060139974
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 40.912506103515625 , std = 19.375179682728103 , Max = 91 , Min = -2 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 74433 Bytes
ratio1= 1.00012207031
ratio2= 0.757171630859
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 30.173248291015625 , std = 14.269045378352326 , Max = 66 , 

In [32]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.0001220703125
   ratio_2= 0.6775639851888021
 
2
   ratio_1= 1.0001220703125
   ratio_2= 0.5606460571289062
 
3
   ratio_1= 1.0001220703125
   ratio_2= 0.6381734212239584
 
4
   ratio_1= 1.0001220703125
   ratio_2= 0.6929092407226562
 
5
   ratio_1= 1.0001220703125
   ratio_2= 0.7160415649414062
 
6
   ratio_1= 1.0001220703125
   ratio_2= 0.6378453572591146
 
7
   ratio_1= 1.0001220703125
   ratio_2= 0.6586964925130209
 
8
   ratio_1= 1.0001220703125
   ratio_2= 0.7352676391601562
 
9
   ratio_1= 1.0001220703125
   ratio_2= 0.6376953125
 
10
   ratio_1= 1.0001220703125
   ratio_2= 0.6583938598632812
 
11
   ratio_1= 1.0001220703125
   ratio_2= 0.6161422729492188
 
12
   ratio_1= 1.0001220703125
   ratio_2= 0.5639165242513021
 
13
   ratio_1= 1.0001220703125
   ratio_2= 0.7030283610026041
 
14
   ratio_1= 1.0001220703125
   ratio_2= 0.7120997111002604
 
15
   ratio_1= 1.0001220703125
   ratio_2= 0.6695988972981771
 
16
   ratio_1= 1.0001220703125
   ratio_2= 0.6128514607